In [0]:
"""
Created on Tue Oct 15 00:14:08 2019

@author: anil
"""

import pandas as pd
import numpy as np
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 
import nltk
nltk.download("popular")
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
!pip install -q keras
import keras
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import Sequential
from keras import layers
from google.colab import drive
drive.mount('/content/drive')
import csv



## Data Reading

In [0]:

"""
Test mode 

"""
olid_training=pd.read_csv("/content/drive/My Drive/OFFENSEVAL20-DATA/offenseval-tr-training-v1.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
X_test_FULL=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/offenseval-tr-testset-v1.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/turkish-goldlabels.tsv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

Pre-processing

In [0]:
"""
	 Converting all text to lowercase 
	 Removing all "\#" symbols
	 Removing all punctuation
	 Removing @user tokens
"""
X_train_FULL.tweet = X_train_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_train_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_train_FULL["tweet_initial"]=filtered_tweets


X_test_FULL.tweet = X_test_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_test_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_test_FULL["tweet_initial"]=filtered_tweets




#FOR TEST

z=[]
for tweet in X_train_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train_FULL["tweet_initial_nontoken"]=z



#FOR TEST

z=[]
for tweet in X_test_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test_FULL["tweet_initial_nontoken"]=z



In [0]:
"""
Importing Keras dependencies / Write Custom Evaluation Metrices
"""



from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
def recall_m(true_Y, pred_Y):
        TP = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        possible_pos = K.sum(K.round(K.clip(true_Y, 0, 1)))
        rec = TP / (possible_pos + K.epsilon())
        return rec

def precision_m(true_Y, pred_Y):
        true_positives = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(pred_Y, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


    

def f1_m(true_Y, pred_Y):
    pres = precision_m(true_Y, pred_Y)
    rec = recall_m(true_Y, pred_Y)
    return 2*((pres*rec)/(pres+rec+K.epsilon()))

from keras.models import Sequential
from keras import layers


## Word2vec / Custom Embedding Layer

In [0]:
fname= "/content/drive/My Drive/Twitter/Word2Vec/w2v_model_word.vec"
import gensim
from gensim.models import Word2Vec
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname)  # you can load this saved keyedvectors model later



## Tokenizing / creating vocabulary and wordindex using keras functinalities

In [0]:
"""
We will use word indexes as look-up table during embedding layer.
"""
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=80202)  #the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
tokenizer.fit_on_texts(X_train_FULL["tweet_initial_nontoken"])
X_train_initial = tokenizer.texts_to_sequences(X_train_FULL["tweet_initial_nontoken"])
X_test_initial = tokenizer.texts_to_sequences(X_test_FULL["tweet_initial_nontoken"])
vocab_size_initial = len(tokenizer.word_index) + 1 
wordIndex_initial=tokenizer.word_index # it is  index
from keras.preprocessing.sequence import pad_sequences
max_len = 50

"""
Padding

"""
X_train_initial = pad_sequences(X_train_initial, padding='post', maxlen=max_len)
X_test_initial = pad_sequences(X_test_initial, padding='post', maxlen=max_len)




In [0]:
 """
Custom Embedding Layer with Word2vec weights
 """

def createEmbeddingLayer(wordIndex,not_static):
  a=[]
  embedding_dim=300
  vocabulary_size=len(wordIndex)+1
  embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
  missed=0
  for word, i in wordIndex.items():
    
          
      try:
          embedding_vector = word_vectors[word] # or fast text
          embedding_matrix[i] = embedding_vector
            

      except KeyError: # If word is not found in the word2vec vocabulary , assign random weights
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)
        missed+=1
        a.append(word)

  print('missed_words :' , missed)

  custom_embedding_layer = Embedding(vocabulary_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                trainable=not_static)
  return custom_embedding_layer

  

In [0]:
  """ CNNLSTM Network architecture  with Keras

  Inputs : Padded Sentences with indexed words ,

  Outputs: Probability and Prediction (Binary) 

  """

  from sklearn.metrics import f1_score
  import pickle

  def CNNLSTM(vocab_size,X_train,X_test,y_train,y_test,wordIndex,trainable):
    early_stopping = [EarlyStopping(monitor='val_loss',
                          min_delta=0,restore_best_weights=True,
                          patience=5,
                          verbose=1, mode='auto')]
    model = Sequential()
    model.add(createFastTextEmbeddingLayer(wordIndex,trainable))
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 1, activation='relu'))
    model.add(Conv1D(128, 3, activation='relu'))

    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(layers.Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False), metrics=['acc',f1_m,precision_m, recall_m])
    ## Fit the model
    model.fit(X_train, y_train, validation_split=0.1, epochs=20,callbacks=early_stopping,batch_size=32)
    loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
    print("LSTM Training Loss: {:.4f}".format(loss))
    print("LSTM Training Accuracy: {:.4f}".format(accuracy))
    print("LSTM Training f1 score: {:.4f}".format(f1_score))
    print("LSTM Training Precision: {:.4f}".format(precision))
    print("LSTM Training Recall: {:.4f}".format(recall))

    loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
    print("LSTM Test Loss: {:.4f}".format(loss))
    print("LSTM Test Accuracy: {:.4f}".format(accuracy))
    print("LSTM Test f1 score: {:.4f}".format(f1_score))
    print("LSTM Test Precision: {:.4f}".format(precision))
    print("LSTM Test Recall: {:.4f}".format(recall))

    probs = model.predict(X_test, verbose=1)
    predicted_classes = model.predict_classes(X_test, verbose=1)


    filename = 'finalized_model_lstm.sav'
    pickle.dump(model, open(filename, 'wb'))

    print(classification_report(y_test, predicted_classes ,digits=3 ))


    print("LSTM  ends..")
    return (predicted_classes,probs)

prediction_cnnlstm,probs_cnnlstm=CNNLSTM(vocab_size_initial,X_train_initial,X_test_initial,Y_TRAIN,Y_TEST_encoded,wordIndex_initial,True)

